# SHAP Values

In [ ]:
!pip install shap

In [ ]:
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import shap
# import keras.backend as K
import json

shap.initjs()

In [ ]:
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
def visualize_model_decisions(shap_values, x, labels=None, figsize=(20, 30)):
    
    colors = []
    for l in np.linspace(1, 0, 100):
        colors.append((30./255, 136./255, 229./255,l))
    for l in np.linspace(0, 1, 100):
        colors.append((255./255, 13./255, 87./255,l))
    red_transparent_blue = LinearSegmentedColormap.from_list("red_transparent_blue", colors)

    multi_output = True
    if type(shap_values) != list:
        multi_output = False
        shap_values = [shap_values]

    # make sure labels
    if labels is not None:
        assert labels.shape[0] == shap_values[0].shape[0], "Labels must have same row count as shap_values arrays!"
        if multi_output:
            assert labels.shape[1] == len(shap_values), "Labels must have a column for each output in shap_values!"
        else:
            assert len(labels.shape) == 1, "Labels must be a vector for single output shap_values."

    # plot our explanations
    fig_size = figsize
    fig, axes = plt.subplots(nrows=x.shape[0], ncols=len(shap_values) + 1, figsize=fig_size)
    if len(axes.shape) == 1:
        axes = axes.reshape(1,axes.size)
    for row in range(x.shape[0]):
        x_curr = x[row].copy()

        # make sure
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 1:
            x_curr = x_curr.reshape(x_curr.shape[:2])
        if x_curr.max() > 1:
            x_curr /= 255.
        
        axes[row,0].imshow(x_curr)
        axes[row,0].axis('off')
        
        # get a grayscale version of the image
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 3:
            x_curr_gray = (0.2989 * x_curr[:,:,0] + 0.5870 * x_curr[:,:,1] + 0.1140 * x_curr[:,:,2]) # rgb to gray
        else:
            x_curr_gray = x_curr

        if len(shap_values[0][row].shape) == 2:
            abs_vals = np.stack([np.abs(shap_values[i]) for i in range(len(shap_values))], 0).flatten()
        else:
            abs_vals = np.stack([np.abs(shap_values[i].sum(-1)) for i in range(len(shap_values))], 0).flatten()
        max_val = np.nanpercentile(abs_vals, 99.9)
        for i in range(len(shap_values)):
            if labels is not None:
                axes[row,i+1].set_title(labels[row,i])
            sv = shap_values[i][row] if len(shap_values[i][row].shape) == 2 else shap_values[i][row].sum(-1)
            axes[row,i+1].imshow(x_curr_gray, cmap=plt.get_cmap('gray'), alpha=0.15, extent=(-1, sv.shape[0], sv.shape[1], -1))
            im = axes[row,i+1].imshow(sv, cmap=red_transparent_blue, vmin=-max_val, vmax=max_val)
            axes[row,i+1].axis('off')
        
    cb = fig.colorbar(im, ax=np.ravel(axes).tolist(), label="SHAP value", orientation="horizontal", aspect=fig_size[0]/0.2)
    cb.outline.set_visible(False)

In [ ]:
model = VGG16(weights="imagenet", include_top=True)
model.summary()

In [ ]:
X, y = shap.datasets.imagenet50()
len(X), len(y)

In [ ]:
import requests

url = "https://images.unsplash.com/photo-1529778873920-4da4926a72c2?ixid=MnwxMjA3fDB8MHxzZWFyY2h8MXx8Y3V0ZSUyMGNhdHxlbnwwfHwwfHw%3D&ixlib=rb-1.2.1&w=1000&q=80"
resp = requests.get(url)

with open('cat.png', 'wb') as f:
  f.write(resp.content)

img_path = 'cat.png'
img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img = keras.preprocessing.image.img_to_array(img)

img.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

to_predict = np.array([X[28], X[35], X[46], img])
fig, ax = plt.subplots(1, 4, figsize=(18, 10))
ax[0].imshow(to_predict[0]/255)
ax[1].imshow(to_predict[1]/255)
ax[2].imshow(to_predict[2]/255)
ax[3].imshow(to_predict[3]/255)



In [ ]:

url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)

In [ ]:
predictions = model.predict(preprocess_input(to_predict.copy()))
predictions, predictions.shape

In [ ]:
predicted_labels = [class_names.get(str(pred)) for pred in np.argmax(predictions, axis=1)]
print(predicted_labels)

In [ ]:
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)

In [ ]:
e = shap.GradientExplainer((model.layers[7].input, model.layers[-1].output), 
                            map2layer(preprocess_input(X.copy()), 7))
shap_values, indexes = e.shap_values(map2layer(to_predict, 7), ranked_outputs=1)
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)
index_names

In [ ]:
visualize_model_decisions(shap_values=shap_values, x=to_predict, 
                          labels=index_names, figsize=(20, 40))

In [ ]:
e = shap.GradientExplainer((model.layers[14].input, model.layers[-1].output), 
                            map2layer(preprocess_input(X.copy()), 14))
shap_values, indexes = e.shap_values(map2layer(to_predict, 14), ranked_outputs=1)
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)
index_names

In [ ]:
visualize_model_decisions(shap_values=shap_values, x=to_predict, 
                          labels=index_names, figsize=(20, 40))

# TF Explain

In [ ]:
!pip install tf-explain

In [ ]:
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
# import tensorflow as tf 
import matplotlib.pyplot as plt
from tf_explain.core.activations import ExtractActivations
from tensorflow.keras.applications.xception import decode_predictions

%matplotlib inline

In [ ]:
model = tf.keras.applications.xception.Xception(weights='imagenet', include_top=True)
model.summary()

In [ ]:
import requests

url = "https://images.unsplash.com/photo-1529778873920-4da4926a72c2?ixid=MnwxMjA3fDB8MHxzZWFyY2h8MXx8Y3V0ZSUyMGNhdHxlbnwwfHwwfHw%3D&ixlib=rb-1.2.1&w=1000&q=80"
resp = requests.get(url)

with open('cat.png', 'wb') as f:
  f.write(resp.content)

img_path = 'cat.png'

In [ ]:
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299))
img = tf.keras.preprocessing.image.img_to_array(img)

plt.imshow(img/255.)

In [ ]:
response = requests.get('https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json')
imgnet_map = response.json()
imgnet_map = {v[1]: k for k, v in imgnet_map.items()}

In [ ]:
img = tf.keras.applications.xception.preprocess_input(img)
predictions = model.predict(np.array([img]))
decode_predictions(predictions, top=5)

In [ ]:
explainer = ExtractActivations()

In [ ]:
grid = explainer.explain((np.array([img]), None), model, ['block1_conv2_act'])
fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(grid, cmap='binary_r')

In [ ]:
grid = explainer.explain((np.array([img]), None), model, ['block2_sepconv2_act'])
fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(grid, cmap='binary_r')

In [ ]:
grid = explainer.explain((np.array([img]), None), model, ['block14_sepconv1_act'])
fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(grid, cmap='binary_r')

# Occlusion Sensitivity

In [ ]:
imgnet_map['Egyptian_cat']

In [ ]:
from tf_explain.core.occlusion_sensitivity import OcclusionSensitivity

explainer = OcclusionSensitivity()

In [ ]:
img_inp = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299))
img_inp = tf.keras.preprocessing.image.img_to_array(img_inp)

In [ ]:
grid = explainer.explain(([img_inp], None), model, 285, 7)
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(grid)

# GradCAM

In [ ]:
from tf_explain.core.grad_cam import GradCAM

In [ ]:
explainer = GradCAM()

In [ ]:
imgnet_map['tabby'], imgnet_map['Egyptian_cat']

In [ ]:
img_inp = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299))
img_inp = tf.keras.preprocessing.image.img_to_array(img_inp)

In [ ]:
grid1 = explainer.explain(
    validation_data=([img], None),
    model=model, layer_name='block1_conv1', class_index=281
)
grid2 = explainer.explain(
    validation_data=([img], None),
    model=model, layer_name='block1_conv1', class_index=285
)

fig = plt.figure(figsize=(18, 8))

ax1 = fig.add_subplot(1, 3, 1)
ax1.imshow(img_inp / 255.)
ax1.imshow(grid1, alpha=0.9)

ax2 = fig.add_subplot(1, 3, 2)
ax2.imshow(img_inp / 255.)
ax2.imshow(grid2, alpha=0.9)

ax3 = fig.add_subplot(1, 3, 3)
ax3.imshow(img_inp / 255. )

In [ ]:
grid1 = explainer.explain(
    validation_data=([img], None),
    model=model, layer_name='block14_sepconv1', class_index=281
)
grid2 = explainer.explain(
    validation_data=([img], None),
    model=model, layer_name='block14_sepconv1', class_index=285
)

fig = plt.figure(figsize=(18, 8))

ax1 = fig.add_subplot(1, 3, 1)
ax1.imshow(img_inp / 255.)
ax1.imshow(grid1, alpha=0.6)

ax2 = fig.add_subplot(1, 3, 2)
ax2.imshow(img_inp / 255.)
ax2.imshow(grid2, alpha=0.6)

ax3 = fig.add_subplot(1, 3, 3)
ax3.imshow(img_inp / 255. )

# Smoothgrad

In [ ]:
from tf_explain.core.smoothgrad import SmoothGrad

explainer = SmoothGrad()


In [ ]:
grid1 = explainer.explain(([img], None), model, 281, 80, .2)
grid2 = explainer.explain(([img], None), model, 285, 80, .2)

fig = plt.figure(figsize=(18, 8))

ax1 = fig.add_subplot(1, 3, 1)
ax1.imshow(img_inp / 255)
ax1.imshow(grid1, alpha=0.9, cmap='binary_r')

ax2 = fig.add_subplot(1, 3, 2)
ax2.imshow(img_inp / 255)
ax2.imshow(grid2, alpha=0.9, cmap="binary_r")

ax3 = fig.add_subplot(1, 3, 3)
ax3.imshow(img_inp / 255)
